# 4.1. Data Cleaning - source Threat Fox

In [1]:
import datetime
import os
import time
import pandas as pd 
import requests
import urllib3
import json
import sys
import numpy as np

pd.options.display.max_colwidth = 1000
pd.set_option('display.max_columns', None)

In [2]:
entries  = r"C:\\Users\\leona\\OneDrive\\Desktop\\Tesi\\Pipeline\\Datasets\\Pipeline\\IocSegnalations\\PostDataStructurization\\Threat_Fox\\Entries.csv"
tags     = r"C:\\Users\\leona\\OneDrive\\Desktop\\Tesi\\Pipeline\\Datasets\\Pipeline\\IocSegnalations\\PostDataStructurization\\Threat_Fox\\Tags.csv"
malwares = r"C:\\Users\\leona\\OneDrive\\Desktop\\Tesi\\Pipeline\\Datasets\\Pipeline\\IocSegnalations\\PostDataStructurization\\Threat_Fox\\Malwares.csv"
alias    = r"C:\\Users\\leona\\OneDrive\\Desktop\\Tesi\\Pipeline\\Datasets\\Pipeline\\IocSegnalations\\PostDataStructurization\\Threat_Fox\\Alias.csv"

## Entries - Threat Fox
### Data Cleaning

#### Error Correction - Missing Values

Drop columns affected by high missings :
- reference

In [3]:
entries_df = pd.read_csv(entries, low_memory = False )
entries_df

,id,ioc,threat_type,ioc_type,malware_printable,malware_malpedia,first_seen,last_seen,reference,reporter
0,1163856,146.56.118.82:443,botnet_cc,ip:port,Cobalt Strike,https://malpedia.caad.fkie.fraunhofer.de/details/win.cobalt_strike,2023-09-15 12:17:38 UTC,NaN,NaN,malpulse
1,1163857,47.94.206.253:8090,botnet_cc,ip:port,Cobalt Strike,https://malpedia.caad.fkie.fraunhofer.de/details/win.cobalt_strike,2023-09-15 12:17:37 UTC,NaN,NaN,malpulse
2,1163858,80.96.156.43:444,botnet_cc,ip:port,Cobalt Strike,https://malpedia.caad.fkie.fraunhofer.de/details/win.cobalt_strike,2023-09-15 12:17:37 UTC,NaN,NaN,malpulse
3,1163859,91.193.75.135:7245,botnet_cc,ip:port,Nanocore RAT,https://malpedia.caad.fkie.fraunhofer.de/details/win.nanocore,2023-09-15 12:17:37 UTC,NaN,NaN,SarlackLab
4,1163860,7245.ddns.net,botnet_cc,domain,Nanocore RAT,https://malpedia.caad.fkie.fraunhofer.de/details/win.nanocore,2023-09-15 12:17:36 UTC,NaN,NaN,SarlackLab
...,...,...,...,...,...,...,...,...,...,...
8186,1155702,41.68.165.218:1177,botnet_cc,ip:port,NjRAT,https://malpedia.caad.fkie.fraunhofer.de/details/win.njrat,2023-09-08 14:45:14 UTC,NaN,NaN,abuse_ch
8187,1155701,41.216.188.29:5200,botnet_cc,ip:port,Ave Maria,https://malpedia.caad.fkie.fraunhofer.de/details/win.ave_maria,2023-09-08 14:00:16 UTC,NaN,NaN,abuse_ch
8188,1155700,193.31.28.123:4444,botnet_cc,ip:port,Meterpreter,https://malpedia.caad.fkie.fraunhofer.de/details/win.meterpreter,2023-09-08 13:35:13 UTC,NaN,NaN,abuse_ch
8189,1155698,162.33.179.240:433,botnet_cc,ip:port,IcedID,https://malpedia.caad.fkie.fraunhofer.de/details/win.icedid,2023-09-08 13:33:51 UTC,NaN,NaN,abuse_ch


In [4]:
entries_df = entries_df.drop([ 'reference'], axis=1)
entries_df

,id,ioc,threat_type,ioc_type,malware_printable,malware_malpedia,first_seen,last_seen,reporter
0,1163856,146.56.118.82:443,botnet_cc,ip:port,Cobalt Strike,https://malpedia.caad.fkie.fraunhofer.de/details/win.cobalt_strike,2023-09-15 12:17:38 UTC,NaN,malpulse
1,1163857,47.94.206.253:8090,botnet_cc,ip:port,Cobalt Strike,https://malpedia.caad.fkie.fraunhofer.de/details/win.cobalt_strike,2023-09-15 12:17:37 UTC,NaN,malpulse
2,1163858,80.96.156.43:444,botnet_cc,ip:port,Cobalt Strike,https://malpedia.caad.fkie.fraunhofer.de/details/win.cobalt_strike,2023-09-15 12:17:37 UTC,NaN,malpulse
3,1163859,91.193.75.135:7245,botnet_cc,ip:port,Nanocore RAT,https://malpedia.caad.fkie.fraunhofer.de/details/win.nanocore,2023-09-15 12:17:37 UTC,NaN,SarlackLab
4,1163860,7245.ddns.net,botnet_cc,domain,Nanocore RAT,https://malpedia.caad.fkie.fraunhofer.de/details/win.nanocore,2023-09-15 12:17:36 UTC,NaN,SarlackLab
...,...,...,...,...,...,...,...,...,...
8186,1155702,41.68.165.218:1177,botnet_cc,ip:port,NjRAT,https://malpedia.caad.fkie.fraunhofer.de/details/win.njrat,2023-09-08 14:45:14 UTC,NaN,abuse_ch
8187,1155701,41.216.188.29:5200,botnet_cc,ip:port,Ave Maria,https://malpedia.caad.fkie.fraunhofer.de/details/win.ave_maria,2023-09-08 14:00:16 UTC,NaN,abuse_ch
8188,1155700,193.31.28.123:4444,botnet_cc,ip:port,Meterpreter,https://malpedia.caad.fkie.fraunhofer.de/details/win.meterpreter,2023-09-08 13:35:13 UTC,NaN,abuse_ch
8189,1155698,162.33.179.240:433,botnet_cc,ip:port,IcedID,https://malpedia.caad.fkie.fraunhofer.de/details/win.icedid,2023-09-08 13:33:51 UTC,NaN,abuse_ch


In [7]:
duplicati = entries_df[entries_df.duplicated(subset=['ioc'], keep=False)]
duplicati

,id,ioc,threat_type,ioc_type,malware_printable,malware_malpedia,first_seen,last_seen,reporter
816,1162951,mycollection-my.xyz,botnet_cc,domain,Lumma Stealer,https://malpedia.caad.fkie.fraunhofer.de/details/win.lumma,2023-09-12 05:19:13 UTC,NaN,500mk500
817,1162952,morevita-my.xyz,botnet_cc,domain,Lumma Stealer,https://malpedia.caad.fkie.fraunhofer.de/details/win.lumma,2023-09-12 05:19:13 UTC,NaN,500mk500
818,1162953,coinflore-my.xyz,botnet_cc,domain,Lumma Stealer,https://malpedia.caad.fkie.fraunhofer.de/details/win.lumma,2023-09-12 05:19:12 UTC,NaN,500mk500
819,1162954,gapi-node.io,botnet_cc,domain,Lumma Stealer,https://malpedia.caad.fkie.fraunhofer.de/details/win.lumma,2023-09-12 05:19:11 UTC,NaN,500mk500
820,1162955,blockall-my.xyz,botnet_cc,domain,Lumma Stealer,https://malpedia.caad.fkie.fraunhofer.de/details/win.lumma,2023-09-12 05:19:10 UTC,NaN,500mk500
821,1162956,westwork-my.xyz,botnet_cc,domain,Lumma Stealer,https://malpedia.caad.fkie.fraunhofer.de/details/win.lumma,2023-09-12 05:19:09 UTC,NaN,500mk500
822,1162957,many-verses.xyz,botnet_cc,domain,Lumma Stealer,https://malpedia.caad.fkie.fraunhofer.de/details/win.lumma,2023-09-12 05:19:09 UTC,NaN,500mk500
823,1162958,fastcloudlife-my.xyz,botnet_cc,domain,Lumma Stealer,https://malpedia.caad.fkie.fraunhofer.de/details/win.lumma,2023-09-12 05:19:08 UTC,NaN,500mk500
824,1162959,ollfiles-my.xyz,botnet_cc,domain,Lumma Stealer,https://malpedia.caad.fkie.fraunhofer.de/details/win.lumma,2023-09-12 05:19:08 UTC,NaN,500mk500
825,1162960,dropfiles-my.xyz,botnet_cc,domain,Lumma Stealer,https://malpedia.caad.fkie.fraunhofer.de/details/win.lumma,2023-09-12 05:19:08 UTC,NaN,500mk500


In [5]:
entries_df[entries_df.ioc == '45.61.138.12:443']

,id,ioc,threat_type,ioc_type,malware_printable,malware_malpedia,first_seen,last_seen,reporter
8190,1155699,45.61.138.12:443,botnet_cc,ip:port,IcedID,https://malpedia.caad.fkie.fraunhofer.de/details/win.icedid,2023-09-08 13:33:51 UTC,2023-09-14 15:33:40 UTC,abuse_ch


Imputation: for 'last_seen' column we could use imputation related to the 'first_seen' column, based on the implicit dependency. I assign the first_seen value if it's missing

In [5]:
entries_df['last_seen'].where(~(entries_df.last_seen.isnull()), other=entries_df.first_seen, inplace=True) 
entries_df

,id,ioc,threat_type,ioc_type,malware_printable,malware_malpedia,first_seen,last_seen,reporter
0,1163856,146.56.118.82:443,botnet_cc,ip:port,Cobalt Strike,https://malpedia.caad.fkie.fraunhofer.de/details/win.cobalt_strike,2023-09-15 12:17:38 UTC,2023-09-15 12:17:38 UTC,malpulse
1,1163857,47.94.206.253:8090,botnet_cc,ip:port,Cobalt Strike,https://malpedia.caad.fkie.fraunhofer.de/details/win.cobalt_strike,2023-09-15 12:17:37 UTC,2023-09-15 12:17:37 UTC,malpulse
2,1163858,80.96.156.43:444,botnet_cc,ip:port,Cobalt Strike,https://malpedia.caad.fkie.fraunhofer.de/details/win.cobalt_strike,2023-09-15 12:17:37 UTC,2023-09-15 12:17:37 UTC,malpulse
3,1163859,91.193.75.135:7245,botnet_cc,ip:port,Nanocore RAT,https://malpedia.caad.fkie.fraunhofer.de/details/win.nanocore,2023-09-15 12:17:37 UTC,2023-09-15 12:17:37 UTC,SarlackLab
4,1163860,7245.ddns.net,botnet_cc,domain,Nanocore RAT,https://malpedia.caad.fkie.fraunhofer.de/details/win.nanocore,2023-09-15 12:17:36 UTC,2023-09-15 12:17:36 UTC,SarlackLab
...,...,...,...,...,...,...,...,...,...
8186,1155702,41.68.165.218:1177,botnet_cc,ip:port,NjRAT,https://malpedia.caad.fkie.fraunhofer.de/details/win.njrat,2023-09-08 14:45:14 UTC,2023-09-08 14:45:14 UTC,abuse_ch
8187,1155701,41.216.188.29:5200,botnet_cc,ip:port,Ave Maria,https://malpedia.caad.fkie.fraunhofer.de/details/win.ave_maria,2023-09-08 14:00:16 UTC,2023-09-08 14:00:16 UTC,abuse_ch
8188,1155700,193.31.28.123:4444,botnet_cc,ip:port,Meterpreter,https://malpedia.caad.fkie.fraunhofer.de/details/win.meterpreter,2023-09-08 13:35:13 UTC,2023-09-08 13:35:13 UTC,abuse_ch
8189,1155698,162.33.179.240:433,botnet_cc,ip:port,IcedID,https://malpedia.caad.fkie.fraunhofer.de/details/win.icedid,2023-09-08 13:33:51 UTC,2023-09-08 13:33:51 UTC,abuse_ch


Imputation: if the field malware_printable is written 'Unknown malware' is set to np.NaN

In [6]:
entries_df['malware_printable'].where(~(entries_df.malware_printable == 'Unknown malware'), other= np.NaN, inplace=True) 

#### Standardization
in the field 'threat_type' we replace botnet_cc into botnet

In [7]:
entries_df['threat_type'].where(~(entries_df.threat_type == 'botnet_cc'), other='botnet', inplace=True) 

- ENTRIES ( // ID_ENTRY, SOURCE // , ioc, ioc_type, threat_type, malware, first_seen, last_seen, reporter, reference )

In [8]:
columns_ordered = ['ID_ENTRY', 'source', 'ioc', 'ioc_type', 'threat_type', 'malware', 'first_seen', 'last_seen', 'reporter', 'reference']
Threat_Fox_ENTRIES_df = entries_df.rename(columns={"id": "ID_ENTRY", "malware_malpedia" : "reference" , 'malware_printable' : 'malware'})
Threat_Fox_ENTRIES_df["source"] = "Threat Fox"
Threat_Fox_ENTRIES_df = Threat_Fox_ENTRIES_df[columns_ordered]

Threat_Fox_ENTRIES_df

,ID_ENTRY,source,ioc,ioc_type,threat_type,malware,first_seen,last_seen,reporter,reference
0,1163856,Threat Fox,146.56.118.82:443,ip:port,botnet,Cobalt Strike,2023-09-15 12:17:38 UTC,2023-09-15 12:17:38 UTC,malpulse,https://malpedia.caad.fkie.fraunhofer.de/details/win.cobalt_strike
1,1163857,Threat Fox,47.94.206.253:8090,ip:port,botnet,Cobalt Strike,2023-09-15 12:17:37 UTC,2023-09-15 12:17:37 UTC,malpulse,https://malpedia.caad.fkie.fraunhofer.de/details/win.cobalt_strike
2,1163858,Threat Fox,80.96.156.43:444,ip:port,botnet,Cobalt Strike,2023-09-15 12:17:37 UTC,2023-09-15 12:17:37 UTC,malpulse,https://malpedia.caad.fkie.fraunhofer.de/details/win.cobalt_strike
3,1163859,Threat Fox,91.193.75.135:7245,ip:port,botnet,Nanocore RAT,2023-09-15 12:17:37 UTC,2023-09-15 12:17:37 UTC,SarlackLab,https://malpedia.caad.fkie.fraunhofer.de/details/win.nanocore
4,1163860,Threat Fox,7245.ddns.net,domain,botnet,Nanocore RAT,2023-09-15 12:17:36 UTC,2023-09-15 12:17:36 UTC,SarlackLab,https://malpedia.caad.fkie.fraunhofer.de/details/win.nanocore
...,...,...,...,...,...,...,...,...,...,...
8186,1155702,Threat Fox,41.68.165.218:1177,ip:port,botnet,NjRAT,2023-09-08 14:45:14 UTC,2023-09-08 14:45:14 UTC,abuse_ch,https://malpedia.caad.fkie.fraunhofer.de/details/win.njrat
8187,1155701,Threat Fox,41.216.188.29:5200,ip:port,botnet,Ave Maria,2023-09-08 14:00:16 UTC,2023-09-08 14:00:16 UTC,abuse_ch,https://malpedia.caad.fkie.fraunhofer.de/details/win.ave_maria
8188,1155700,Threat Fox,193.31.28.123:4444,ip:port,botnet,Meterpreter,2023-09-08 13:35:13 UTC,2023-09-08 13:35:13 UTC,abuse_ch,https://malpedia.caad.fkie.fraunhofer.de/details/win.meterpreter
8189,1155698,Threat Fox,162.33.179.240:433,ip:port,botnet,IcedID,2023-09-08 13:33:51 UTC,2023-09-08 13:33:51 UTC,abuse_ch,https://malpedia.caad.fkie.fraunhofer.de/details/win.icedid


### Normalization
All the chars of malwares are lower cases and all the punctualization factors are deleted

In [9]:
import math

In [10]:
def prepare_for_similarity_comparison(df, col_i, col_f):
    
    for i in range(len(df)):
        
        value = df.at[i,col_i]

        if (not isinstance(value, float) or not math.isnan(value)):    
            
            value = ' ' + value + ' ' 
            value = value.lower() 
            value = value.replace('?', ' ').replace('!', ' ').replace('%', ' ').replace('.', ' ').replace('-', ' ').replace('[' , ' ').replace( ']', ' ').replace('(' , ' ').replace( ')', ' ').replace('\\',' ').replace( ',', ' ').replace( '_', ' ') 
            value = value.strip() 
        
        df.loc[i, col_f] = value
            
    return df

In [11]:
Threat_Fox_ENTRIES_df = prepare_for_similarity_comparison(Threat_Fox_ENTRIES_df, 'malware', 'malware')
Threat_Fox_ENTRIES_df

,ID_ENTRY,source,ioc,ioc_type,threat_type,malware,first_seen,last_seen,reporter,reference
0,1163856,Threat Fox,146.56.118.82:443,ip:port,botnet,cobalt strike,2023-09-15 12:17:38 UTC,2023-09-15 12:17:38 UTC,malpulse,https://malpedia.caad.fkie.fraunhofer.de/details/win.cobalt_strike
1,1163857,Threat Fox,47.94.206.253:8090,ip:port,botnet,cobalt strike,2023-09-15 12:17:37 UTC,2023-09-15 12:17:37 UTC,malpulse,https://malpedia.caad.fkie.fraunhofer.de/details/win.cobalt_strike
2,1163858,Threat Fox,80.96.156.43:444,ip:port,botnet,cobalt strike,2023-09-15 12:17:37 UTC,2023-09-15 12:17:37 UTC,malpulse,https://malpedia.caad.fkie.fraunhofer.de/details/win.cobalt_strike
3,1163859,Threat Fox,91.193.75.135:7245,ip:port,botnet,nanocore rat,2023-09-15 12:17:37 UTC,2023-09-15 12:17:37 UTC,SarlackLab,https://malpedia.caad.fkie.fraunhofer.de/details/win.nanocore
4,1163860,Threat Fox,7245.ddns.net,domain,botnet,nanocore rat,2023-09-15 12:17:36 UTC,2023-09-15 12:17:36 UTC,SarlackLab,https://malpedia.caad.fkie.fraunhofer.de/details/win.nanocore
...,...,...,...,...,...,...,...,...,...,...
8186,1155702,Threat Fox,41.68.165.218:1177,ip:port,botnet,njrat,2023-09-08 14:45:14 UTC,2023-09-08 14:45:14 UTC,abuse_ch,https://malpedia.caad.fkie.fraunhofer.de/details/win.njrat
8187,1155701,Threat Fox,41.216.188.29:5200,ip:port,botnet,ave maria,2023-09-08 14:00:16 UTC,2023-09-08 14:00:16 UTC,abuse_ch,https://malpedia.caad.fkie.fraunhofer.de/details/win.ave_maria
8188,1155700,Threat Fox,193.31.28.123:4444,ip:port,botnet,meterpreter,2023-09-08 13:35:13 UTC,2023-09-08 13:35:13 UTC,abuse_ch,https://malpedia.caad.fkie.fraunhofer.de/details/win.meterpreter
8189,1155698,Threat Fox,162.33.179.240:433,ip:port,botnet,icedid,2023-09-08 13:33:51 UTC,2023-09-08 13:33:51 UTC,abuse_ch,https://malpedia.caad.fkie.fraunhofer.de/details/win.icedid


## Tags - Threat Fox
### Data Cleaning

#### Error Correction

All the 3 attributes are keys of the entity, so if one of the them is Nan, the row is dropped

In [12]:
tags_df = pd.read_csv(tags, low_memory = False )
tags_df

,ID_ENTRY,source,tag
0,1155537,Threat Fox,redline_stealer
1,1155537,Threat Fox,infostealer
2,1155537,Threat Fox,stealer
3,1155538,Threat Fox,redline_stealer
4,1155538,Threat Fox,RAT
...,...,...,...
25034,1163859,Threat Fox,RAT
25035,1163860,Threat Fox,RAT
25036,1163860,Threat Fox,NanoCore
25037,1163861,Threat Fox,RemcosRAT


In [13]:
Threat_Fox_TAGS_df = tags_df.dropna().reset_index(drop = True)

#### Data Normalization 

All the chars of tag are lower cases and all the punctualization factors are deleted

In [14]:
Threat_Fox_TAGS_df = prepare_for_similarity_comparison(Threat_Fox_TAGS_df, 'tag', 'tag')
Threat_Fox_TAGS_df

,ID_ENTRY,source,tag
0,1155537,Threat Fox,redline stealer
1,1155537,Threat Fox,infostealer
2,1155537,Threat Fox,stealer
3,1155538,Threat Fox,redline stealer
4,1155538,Threat Fox,rat
...,...,...,...
24742,1163859,Threat Fox,rat
24743,1163860,Threat Fox,rat
24744,1163860,Threat Fox,nanocore
24745,1163861,Threat Fox,remcosrat


#### Data Deduplications 

Drop duplicates

In [15]:
Threat_Fox_TAGS_df = Threat_Fox_TAGS_df.drop_duplicates().reset_index(drop = True)
Threat_Fox_TAGS_df

,ID_ENTRY,source,tag
0,1155537,Threat Fox,redline stealer
1,1155537,Threat Fox,infostealer
2,1155537,Threat Fox,stealer
3,1155538,Threat Fox,redline stealer
4,1155538,Threat Fox,rat
...,...,...,...
24742,1163859,Threat Fox,rat
24743,1163860,Threat Fox,rat
24744,1163860,Threat Fox,nanocore
24745,1163861,Threat Fox,remcosrat


## Malwares - Threat Fox
### Data Cleaning

#### Error Correction


In [16]:
malwares_df = pd.read_csv(malwares, low_memory = False )
malwares_df

,malware,type
0,Cobalt Strike,unknown
1,Nanocore RAT,unknown
2,Remcos,unknown
3,Loki Password Stealer (PWS),unknown
4,DCRat,unknown
...,...,...
63,Loda,unknown
64,SpyNote,unknown
65,SectopRAT,unknown
66,Ghost RAT,unknown


In [17]:
Threat_Fox_MALWARES_df = malwares_df.dropna().reset_index(drop = True)

#### Data Normalization 

All the chars of tag are lower cases and all the punctualization factors are deleted

In [18]:
Threat_Fox_MALWARES_df = prepare_for_similarity_comparison(Threat_Fox_MALWARES_df, 'malware', 'malware')
Threat_Fox_MALWARES_df

,malware,type
0,cobalt strike,unknown
1,nanocore rat,unknown
2,remcos,unknown
3,loki password stealer pws,unknown
4,dcrat,unknown
...,...,...
63,loda,unknown
64,spynote,unknown
65,sectoprat,unknown
66,ghost rat,unknown


#### Data Deduplications 

Drop duplicates

In [19]:
Threat_Fox_MALWARES_df = Threat_Fox_MALWARES_df.drop_duplicates()
Threat_Fox_MALWARES_df = Threat_Fox_MALWARES_df.reset_index(drop = True)
Threat_Fox_MALWARES_df

,malware,type
0,cobalt strike,unknown
1,nanocore rat,unknown
2,remcos,unknown
3,loki password stealer pws,unknown
4,dcrat,unknown
...,...,...
63,loda,unknown
64,spynote,unknown
65,sectoprat,unknown
66,ghost rat,unknown


## Alias - Threat Fox
### Data Cleaning

#### Error Correction


In [20]:
alias_df = pd.read_csv(alias, low_memory = False )
alias_df

,malware,alias
0,Cobalt Strike,Agentemis
1,Nanocore RAT,Nancrat
2,Remcos,RemcosRAT
3,Loki Password Stealer (PWS),Burkina
4,DCRat,DarkCrystal RAT
...,...,...
171,Houdini,WSHRAT
172,SmokeLoader,NaN
173,Ave Maria,NaN
174,Houdini,dinihou


In [21]:
Threat_Fox_ALIAS_df = alias_df.dropna().reset_index(drop = True)

#### Data Normalization 

All the chars of tag are lower cases and all the punctualization factors are deleted

In [22]:
Threat_Fox_ALIAS_df = prepare_for_similarity_comparison(Threat_Fox_ALIAS_df, 'malware', 'malware')
Threat_Fox_ALIAS_df = prepare_for_similarity_comparison(Threat_Fox_ALIAS_df, 'alias', 'alias')

Threat_Fox_ALIAS_df

,malware,alias
0,cobalt strike,agentemis
1,nanocore rat,nancrat
2,remcos,remcosrat
3,loki password stealer pws,burkina
4,dcrat,darkcrystal rat
...,...,...
71,smokeloader,smoke loader
72,ave maria,avemaria
73,houdini,wshrat
74,houdini,dinihou


#### Data Deduplications 

Drop duplicates

In [23]:
Threat_Fox_ALIAS_df = Threat_Fox_ALIAS_df.drop_duplicates().reset_index(drop = True)
Threat_Fox_ALIAS_df

,malware,alias
0,cobalt strike,agentemis
1,nanocore rat,nancrat
2,remcos,remcosrat
3,loki password stealer pws,burkina
4,dcrat,darkcrystal rat
...,...,...
71,smokeloader,smoke loader
72,ave maria,avemaria
73,houdini,wshrat
74,houdini,dinihou


# SAVE into ..\IocSegnalations\PostDataCleaning\Threat_Fox\ folder


In [24]:
dataframes_dict = {}

# Add the DataFrames to the dictionary with keys
dataframes_dict['Entries']  = Threat_Fox_ENTRIES_df
dataframes_dict['Tags']     = Threat_Fox_TAGS_df
dataframes_dict['Malwares'] = Threat_Fox_MALWARES_df
dataframes_dict['Alias']    = Threat_Fox_ALIAS_df

In [26]:
save_directory = "C:\\Users\\leona\\OneDrive\\Desktop\\Tesi\\Pipeline\\Datasets\\Pipeline\\IocSegnalations\\PostDataCleaning\\Threat_Fox\\"
for key, df in dataframes_dict.items():
    file_name = f"{key}.csv"
    file_path = save_directory + file_name
    df.to_csv(file_path, index=False)